In [2]:
!ls

Business_Understanding.ipynb           Data_Understanding-V1.1.ipynb
Data_Preparation_Large_File_V1.1.ipynb Data_Understanding_EDA_1.0.ipynb
Data_Preparation_V1.1.ipynb


In [6]:
import subprocess
import os

import pandas as pd

import requests
from bs4 import BeautifulSoup

import json

pd.set_option('display.max_rows', 500)

![CRISP_DM](../reports/figures/CRISP_DM.png)

# Data Understanding

* RKI, webscrape (webscraping) https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html
* John Hopkins (GITHUB) https://github.com/CSSEGISandData/COVID-19.git
* REST API services to retreive data https://npgeo-corona-npgeo-de.hub.arcgis.com/

## GITHUB csv data

git clone/pull https://github.com/CSSEGISandData/COVID-19.git

In [7]:

git_pull = subprocess.Popen( "/usr/bin/git pull" , 
                     cwd = os.path.dirname( '../data/raw/COVID-19/' ), 
                     shell = True, 
                     stdout = subprocess.PIPE, 
                     stderr = subprocess.PIPE )
(out, error) = git_pull.communicate()


print("Error : " + str(error)) 
print("out : " + str(out))

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/COVID-19'

In [5]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'

In [4]:
pd_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,555,607,665,714,784,840,906,933,996,1026
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,433,446,467,475,494,518,539,548,562,584
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,1825,1914,1983,2070,2160,2268,2418,2534,2629,2718
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,601,638,646,659,673,673,696,704,713,717
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,19,19,19,19,19,19,19,24,24,24


## Webscrapping

In [5]:
page = requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [6]:
soup = BeautifulSoup(page.content, 'html.parser')

In [7]:
html_table=soup.find('table') # find the table, attention this works if one table exists

In [8]:
all_rows=html_table.find_all('tr')

In [9]:
final_data_list=[]

In [10]:
for pos,rows in enumerate(all_rows):
   
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')] #td for data element
    final_data_list.append(col_list)
    

In [11]:
pd_daily_status=pd.DataFrame(final_data_list).dropna().rename(columns={0:'state',
                                                       1:'cases',
                                                       2:'changes',
                                                       3:'cases_per_100k',
                                                       4:'fatal',
                                                       5:'comment'})

In [12]:
pd_daily_status.head()

,state,cases,changes,cases_per_100k,fatal
2,Baden-Württem­berg,28.712,+459,259,1.031
3,Bayern,38.310,+461,293,1.336
4,Berlin,5.237,+41,140,97
5,Brandenburg,2.275,+37,91,67
6,Bremen,609,+5,89,25


## REST API calls

In [13]:
## data request for Germany
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [14]:
json_object=json.loads(data.content) 

In [15]:
type(json_object)

dict

In [16]:
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'serverGens', 'geometryType', 'spatialReference', 'fields', 'features'])

In [17]:
full_list=[]
for pos,each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])
    

In [18]:
pd_full_list=pd.DataFrame(full_list)
pd_full_list.head()

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death
0,1,01,Schleswig-Holstein,Land,2896712,15,2414,1587420000000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,83.335865,4.573731e+10,2.881496e+06,70
1,2,02,Hamburg,Freie und Hansestadt,1841179,6,4204,1587420000000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,228.331955,2.089396e+09,4.188002e+05,91
2,3,03,Niedersachsen,Land,7982448,9,9098,1587420000000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,113.975061,1.299836e+11,4.008988e+06,306
3,4,04,Bremen,Freie Hansestadt,682986,5,609,1587420000000,04,4132268b-54de-4327-ac1e-760e915112f1,89.167274,1.119157e+09,3.357177e+05,25
4,5,05,Nordrhein-Westfalen,Land,17932651,10,29389,1587420000000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,163.885418,8.782936e+10,2.648673e+06,896


In [19]:

pd_full_list.to_csv('../data/raw/NPGEO/GER_state_data.csv',sep=';')

In [20]:
pd_full_list.shape[0]

16

# API access via REST service, e.g. USA data 

example of a REST conform interface (attention registration mandatory)

www.smartable.ai

In [21]:


# US for full list
headers = {
    'Cache-Control': 'no-cache',
    'Subscription-Key': '28ee4219700f48718be78b057beb7eb4',
}

response = requests.get('https://api.smartable.ai/coronavirus/stats/US', headers=headers)
print(response)

<Response [200]>


In [22]:

US_dict=json.loads(response.content) # imports string
with open('../data/raw/SMARTABLE/US_data.json', 'w') as outfile:
    json.dump(US_dict, outfile,indent=2)

In [23]:
print(json.dumps(US_dict,indent=2)) #string dump

{
  "location": {
    "long": -95.712891,
    "countryOrRegion": "United States",
    "provinceOrState": null,
    "county": null,
    "isoCode": "US",
    "lat": 37.09024
  },
  "updatedDateTime": "2020-04-21T06:45:14.5523523Z",
  "stats": {
    "totalConfirmedCases": 789584,
    "newlyConfirmedCases": 28176,
    "totalDeaths": 42451,
    "newDeaths": 1770,
    "totalRecoveredCases": 67605,
    "newlyRecoveredCases": 1872,
    "history": [
      {
        "date": "2020-01-22T00:00:00",
        "confirmed": 1,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-23T00:00:00",
        "confirmed": 1,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-24T00:00:00",
        "confirmed": 2,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-25T00:00:00",
        "confirmed": 2,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-26T00:00:00",
        "con

# Individual States US

In [24]:
US_dict['stats']['breakdowns'][0]

{'location': {'long': -86.902298,
  'countryOrRegion': 'United States',
  'provinceOrState': 'Alabama',
  'county': None,
  'isoCode': 'US-AL',
  'lat': 32.318231},
 'totalConfirmedCases': 5078,
 'newlyConfirmedCases': 175,
 'totalDeaths': 164,
 'newDeaths': 4,
 'totalRecoveredCases': 0,
 'newlyRecoveredCases': 0}

In [25]:
full_list_US_country=[]
for pos,each_dict in enumerate (US_dict['stats']['breakdowns'][:]):
    flatten_dict=each_dict['location']
    flatten_dict.update(dict(list(US_dict['stats']['breakdowns'][pos].items())[1: 7]) 
    )
    full_list_US_country.append(flatten_dict)

In [26]:
pd.DataFrame(full_list_US_country).to_csv('../data/raw/SMARTABLE/full_list_US_country.csv',sep=';',index=False)